In [1]:
import pandas as pd
import os
from pyomo.environ import *
from pyomo.opt import SolverFactory
os.chdir(r"C:\Users\User\Desktop\BUS 730 - Prescriptive Modeling & Optimization for BA\Assignments\HW2_Linear Optimization") 

In [2]:
# Load in data (located on the first sheet)
df = pd.read_excel(open('MP_scenarios.xlsx','rb'), sheet_name='Sheet1') 
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
0,1,55.48,120,160,194,223,15,1421,13.870
1,2,53.68,115,149,171,197,15,2396,13.420
2,3,61.56,140,191,207,459,15,2544,15.390
3,4,65.72,115,151,242,278,15,1316,16.430
4,5,64.98,120,173,221,335,15,1377,16.245


In [3]:
# Write code that solves one instance of the markdown pricing problem

inputs = [1,55.48,120,160,194,223,15,1421,13.870] 
discount = [0, 0.15, 0.3, 0.5]   # discount levels (0%, 15%, 30%, 50%)

discounted_price = []
for i in discount:
    discounted_price.append(inputs[1]*(1-i))

# declare a concrete model
model = ConcreteModel()

# declare the decision variables (the number of weeks at each discount level) 
num_discount = len(discount) # the number of decision variables 
model.x = Var(range(num_discount), domain = NonNegativeReals)  

# specify the objective: maximize total revenues (sales + salvage)

sold_units = sum(model.x[i]*inputs[i+2] for i in range(num_discount))   # number of weeks * demand rate
sales = sum((model.x[i]*inputs[i+2])*discounted_price[i] for i in range(num_discount)) # sum of (units sold * unit price)
salvaged_units = inputs[-2] - sold_units   # inventory remaining - units sold
                 
model.Objective = Objective(expr = sales + (salvaged_units*inputs[-1]), sense= maximize)  # maximize total revenues

# set the invertory available for sale constraint (<= inventory remaining)
model.InvertoryConstraint = Constraint(expr = sold_units <= inputs[-2])  

# set the selling weeks constraint (<= weeks left)
model.WeekConstraint = Constraint(expr = sum(model.x[i] for i in range(num_discount)) <= inputs[-3])  
    
# specify the solver and solve
opt = SolverFactory('glpk')
results = opt.solve(model)#,tee = True)

In [4]:
# Print out the solution

pricing_plan = []  # number of weeks at each discount level (DVs)
price_levels = []
for i in range(num_discount):
    pricing_plan.append(model.x[i]())
    if (model.x[i]()!=0):    # number of weeks is not 0
        price_levels.append(discounted_price[i])

print("The optimized pricing plan:", pricing_plan)
print("Revenue projection:", model.Objective())
print("Current price:", price_levels[0])

solution = []
solution.append(pricing_plan)       # optimized pricing plan
solution.append(model.Objective())  # total revenue projection
solution.append(price_levels[0])    # current price level according to the optimized pricing plan
print(solution)

The optimized pricing plan: [11.8416666666667, 0.0, 0.0, 0.0]
Revenue projection: 78837.08000000016
Current price: 55.48
[[11.8416666666667, 0.0, 0.0, 0.0], 78837.08000000016, 55.48]


In [5]:
# Wrap the code above into a function that takes each product data as an input and ouputs the solution

def solvepricing(inputs):
    discount = [0, 0.15, 0.3, 0.5]   # discount levels (0%, 15%, 30%, 50%)

    discounted_price = []
    for i in discount:
        discounted_price.append(inputs[1]*(1-i))

    # declare a concrete model
    model = ConcreteModel()

    # declare the decision variables (the number of weeks at each discount level) 
    num_discount = len(discount) # the number of decision variables 
    model.x = Var(range(num_discount), domain = NonNegativeReals)  

    # specify the objective: maximize total revenues (sales + salvage)

    sold_units = sum(model.x[i]*inputs[i+2] for i in range(num_discount))   # number of weeks * demand rate
    sales = sum((model.x[i]*inputs[i+2])*discounted_price[i] for i in range(num_discount)) # sum of (units sold * unit price)
    salvaged_units = inputs[-2] - sold_units   # inventory remaining - units sold

    model.Objective = Objective(expr = sales + (salvaged_units*inputs[-1]), sense= maximize)  # maximize total revenues

    # set the invertory available for sale constraint (<= inventory remaining)
    model.InvertoryConstraint = Constraint(expr = sold_units <= inputs[-2])  

    # set the selling weeks constraint (<= weeks left)
    model.WeekConstraint = Constraint(expr = sum(model.x[i] for i in range(num_discount)) <= inputs[-3])  

    # specify the solver and solve
    opt = SolverFactory('glpk')
    results = opt.solve(model)#,tee = True)
    
    # output the solution
    pricing_plan = []  # number of weeks at each discount level (DVs)
    price_levels = []
    for i in range(num_discount):
        pricing_plan.append(model.x[i]())
        if (model.x[i]()!=0):    # number of weeks is not 0
            price_levels.append(discounted_price[i])

    solution = []
    solution.append(pricing_plan)       # optimized pricing plan
    solution.append(model.Objective())  # total revenue projection
    solution.append(price_levels[0])    # current price level according to the optimized pricing plan
    return solution

In [6]:
# Check the function
solvepricing([3,61.56,140,191,207,459,15,2544,15.390])

[[6.29411764705882, 8.70588235294118, 0.0, 0.0], 141254.12752941175, 61.56]

In [7]:
# Run a loop on the data in df to solve all the problems in the dataset

# First, do it for one value of k to make sure it works
k = 0 # use k to index the row
inputs = df.iloc[k].values.tolist()
solvepricing(inputs) # it works!

[[11.8416666666667, 0.0, 0.0, 0.0], 78837.08000000016, 55.48]

In [8]:
# Now put it in a loop to solve all the problems in the dataset

outputs = []
for k in range(len(df)):
    inputs = df.iloc[k].values.tolist()
    outputs.append(solvepricing(inputs))

# add it back to dataframe
output1 = []
output2 = []
output3 = []
for i in range(len(df)):
    output1.append(outputs[i][0])
    output2.append(outputs[i][1])
    output3.append(outputs[i][2])
    
df['Optimized Pricing Plan (0%, 15%, 30%, 50%)'] = output1  # the number of weeks at each discount level 
df['Revenue Projection'] = output2
df['Current Price'] = output3
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value,"Optimized Pricing Plan (0%, 15%, 30%, 50%)",Revenue Projection,Current Price
0,1,55.48,120,160,194,223,15,1421,13.870,"[11.8416666666667, 0.0, 0.0, 0.0]",78837.080000,55.480
1,2,53.68,115,149,171,197,15,2396,13.420,"[0.0, 15.0, 0.0, 0.0]",104139.200000,45.628
2,3,61.56,140,191,207,459,15,2544,15.390,"[6.29411764705882, 8.70588235294118, 0.0, 0.0]",141254.127529,61.560
3,4,65.72,115,151,242,278,15,1316,16.430,"[11.4434782608696, 0.0, 0.0, 0.0]",86487.520000,65.720
4,5,64.98,120,173,221,335,15,1377,16.245,"[11.475, 0.0, 0.0, 0.0]",89477.460000,64.980


In [9]:
# Export df to csv
df.to_csv("MP_scenarios_solution.csv", index=False)